## Why Shifting to PyTorch?
- Because of High Compute Time in Manual Looping and Memory Allocation

## What will change?
- Just the Core Classes but built on top of nn.Module()

## Same Dataset used as earlier

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Load all the words from the '.txt' file
words = open('names.txt', mode = 'r', encoding='utf-8').read().splitlines()

# Encoder and Decoder
# A little bit change in the Token Mapping
# I'll use:
# - 0 to denote ---> <PAD> tokens
# - 1 to denote ---> '.' (end token)
# - and all characters from [2, .., 28]

chars = sorted(list(set(''.join(words))))

# Reserve 0 for PAD token
stoi = {c: i+2 for i, c in enumerate(chars)}  # alphabet starts at index 2
stoi['.'] = 1   # END token
stoi['<PAD>'] = 0   # padding token
itos = {i: c for c, i in stoi.items()}

def encode_word(word, stoi):
    return [stoi[c] for c in word] + [stoi['.']]

def decode_indices(indices, itos):
    return [itos[i.item()] for i in indices]

encoded = [encode_word(w, stoi) for w in words]
max_len = max(len(seq) for seq in encoded)

def pad_sequences(sequences, max_len):
    X, Y = [], []

    for seq in sequences:
        x = seq[:-1]                        # all but last char
        y = seq[1:]                         # all but first char

        # padding
        x += [0] * (max_len - len(x))
        y += [-100] * (max_len - len(y))

        X.append(x)
        Y.append(y)

    return torch.tensor(X), torch.tensor(Y)

def generate_dataset():
    words = open('names.txt', mode = 'r', encoding='utf-8').read().splitlines()
    encoded = [encode_word(w, stoi) for w in words]
    max_len = max(len(seq) for seq in encoded)
    x_tensor, y_tensor = pad_sequences(encoded, max_len)
    return x_tensor.long(), y_tensor.long()

In [2]:
from sklearn.model_selection import train_test_split
X, y = generate_dataset()
x_train, x_val, y_train, y_val = train_test_split(
    X, y, test_size = 0.2)

## GRUCell's Implementation

In [3]:
class GRUCell(nn.Module):
    """ 
    This Implementation is Differ from the PyTorch's Official Implementation in 2 Different Ways
    NOTE-1:
        This is not the official Implementation of PyTorch's GRUcell Because they use 2 biases per Gate
        and i'm only using 1 bias per Gate

        PyTorch's Official Implementation: 
        r = σ(W_ir x + b_ir + W_hr h + b_hr)
        z = σ(W_iz x + b_iz + W_hz h + b_hz)
        n = tanh(W_in x + b_in + r ⊙ (W_hn h + b_hn))

        They Use 2 Bias per Gate
    
    NOTE-2: 
        They apply the reset gate (r) after the Multiplication of W_hn and addition of b_hn on the h_prev
        2. Original implementation: Apply the Hadamard product (⊙) between r_t and h_prev and then apply the
            Matrix Transformation and bias addition
        3. What PyTorch does is, they apply the Matrix Transformation (Matrix Multiplication and bias addition) 1st and then
            they apply the Hadamard product (⊙) between (W_hn h + b_hn)
    """
    def __init__(self, embd_dim, hidden_dim):
        """
        -> Bias only on x: input
        -> No Bias on Hidden States
        """
        super().__init__()
        self.embd_dim = embd_dim
        self.hidden_dim = hidden_dim

        # Candidate transformation
        self.Wx = nn.Linear(embd_dim, hidden_dim, bias = True)
        self.Wh = nn.Linear(hidden_dim, hidden_dim, bias = False)

        # Update Gate Specific Parameters
        self.Wzx = nn.Linear(embd_dim, hidden_dim, bias = True)
        self.Wzh = nn.Linear(hidden_dim, hidden_dim, bias = False)
        self.bias_z = nn.Parameter(torch.zeros(hidden_dim))

        # Reset Gate Specific Parameters
        self.Wrx = nn.Linear(embd_dim, hidden_dim, bias = True)
        self.Wrh = nn.Linear(hidden_dim, hidden_dim, bias = False)
        self.bias_r = nn.Parameter(torch.zeros(hidden_dim))

    def forward(self, x, h_prev):
        """
        --> A proposed update → candidate (h̃_t)
        --> A decision gate → update gate (z_t)
        --> A final controlled update → h_t

        NOTE:   1. Reset Gate Filters h_prev
                    - r_t = sigmoid( ( (x_t @ W_rx) + (h_prev @ W_rh) + b_r) )
                2. Apply filter to h_prev
                    - filtered_h_prev = r_t * h_prev [NOTE: (where * is element-wise multiplication)]
                        - Meaning:
                            - If r_t ≈ 0 → ignore old memory when forming candidate
                            - If r_t ≈ 1 → use old memory fully
                3. Compute candidate
                    - h̃_t = tanh( ( (x_t @ W_hx) + (filtered_h_prev @ W_hh) + b_h) )
                        - Meaning: 
                            - This produces a new memory proposal: A proposed update
                4. Final hidden state
                    - h_t = (1 - z_t) * h_prev + z_t * h̃_t [NOTE: (where * is element-wise multiplication)]
        """
        r_t = torch.sigmoid((self.Wrx(x)) + (self.Wrh(h_prev)) + self.bias_r)
        z_t = torch.sigmoid((self.Wzx(x)) + (self.Wzh(h_prev)) + self.bias_z)
        h_tilde = torch.tanh(self.Wx(x) + self.Wh(r_t * h_prev))

        h = (1 - z_t) * h_prev + z_t * h_tilde
        return h


## GRULayer's Implementation

In [4]:
class GRULayer(nn.Module):
    def __init__(self, embd_dim, hidden_dim, dropout = 0.0):
        super().__init__()
        self.grucell = GRUCell(embd_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, h_prev = None):
        batch, seq_length, _ = x.shape # x.shape --> batch, seq_length, embd_dim
        if h_prev is None:
            h_prev = torch.zeros(batch, self.grucell.hidden_dim, device = x.device)

        hidden_states = []
        for t in range(seq_length):
            x_t = x[:, t, :]
            h_prev = self.grucell(x_t, h_prev)
            h_prev = self.dropout(h_prev)
            hidden_states.append(h_prev)
        
        # Stack list into tensor
        hidden_states = torch.stack(hidden_states, dim=1)
        return hidden_states
        

## Linear Layer Projection

In [5]:
class Linear(nn.Module):
    def __init__(self, hidden_dim, n_classes):
        """
        This layer performs a linear projection on the GRU hidden states.
        It maps the hidden vector (of size hidden_dim) into the vocabulary space (n_classes)
        by applying a learnable affine transformation:

            logits = W h + b

        This is used to convert each GRU hidden state into class probabilities
        (e.g., next-character prediction in a name generation model).
        """

        super().__init__()
        self.linear_projection = nn.Linear(in_features = hidden_dim, out_features = n_classes, bias = True)
    
    def forward(self, x):
        return self.linear_projection(x)

# GRU Model Family: Architectural Variants and Hyperparameter Configurations

In this section, three distinct GRU-based architectures are defined for the task of character-level name generation.
Each model represents a different point on the capacity–complexity spectrum, enabling controlled experiments regarding sequence modeling depth, representational power, and generalization.
The following subsections document each model variant, its architectural design, and the rationale behind the selected hyperparameters.

---

## 1. GRU-Small Model

### Architectural Description

The small variant is a lightweight single-layer GRU architecture designed primarily for baseline performance measurement and rapid experimentation.
It consists of:

* An embedding layer mapping discrete character indices to continuous vectors.
* A single GRU layer operating over sequences of embeddings.
* A linear projection layer mapping hidden states to vocabulary logits.

### Hyperparameter Configuration

```
embedding_dim = 64
hidden_dim    = 128
epochs        = 35
learning_rate = 1e-3
batch_size    = 128
block_size    = 12
dropout       = 0.0
optimizer     = Adam
```

### Purpose and Expected Behavior

This model is suitable for preliminary studies, small datasets, debugging the training loop, and validating convergence behavior.
Its capacity is intentionally constrained to expose underfitting tendencies and provide a clean comparison baseline for larger models.

---

## 2. GRU-Medium Model

### Architectural Description

The medium-sized architecture increases modeling depth by stacking two GRU layers.
The output of the first GRU layer serves as the input to the second, enabling the model to capture more complex sequential patterns and longer-range dependencies.
It includes:

* Embedding layer
* GRU Layer 1
* GRU Layer 2
* Linear output layer

### Hyperparameter Configuration

```
embedding_dim = 96
hidden_dim    = 256
epochs        = 40
learning_rate = 5e-4
batch_size    = 128
block_size    = 16
dropout       = 0.1
optimizer     = Adam
```

### Purpose and Expected Behavior

This configuration balances computational cost and expressive power.
It is recommended for medium-sized datasets (20k–50k sequences) and typically exhibits substantially improved name-generation quality over the single-layer model, with a reduced risk of underfitting.

---

## 3. GRU-Large Model

### Architectural Description

The large variant maximizes modeling capacity by stacking three GRU layers and applying layer normalization before the final projection.
This design improves gradient stability in deep recurrent stacks and enhances pattern extraction across long context windows.
The model consists of:

* Embedding layer
* GRU Layer 1
* GRU Layer 2
* GRU Layer 3
* LayerNorm for stabilization
* Linear projection head

### Hyperparameter Configuration

```
embedding_dim = 128
hidden_dim    = 512
epochs        = 60
learning_rate = 3e-4
batch_size    = 256
block_size    = 20
dropout       = 0.2
optimizer     = AdamW
```

### Purpose and Expected Behavior

This model is designed for large-scale experiments where high-quality sequence generation is essential.
It is well suited for datasets with more than 30k examples (e.g., your dataset with 32k names).
The combination of deeper recurrence, higher dimensionality, and AdamW regularization typically yields the strongest generalization and sampling quality.

---

# Summary

The three GRU architectures—Small, Medium, and Large—provide a systematic framework for analyzing how increasing representational depth and parameter count influence performance on name-generation tasks.
These models are meant to be trained independently and compared using quantitative metrics (training and validation loss curves) and qualitative evaluation (sampled names).

In [6]:
class GRU_SmallModel(nn.Module):
    def __init__(self, n_classes, embd_dim, hidden_dim, dropout = 0.0):
        """
        I'll Train this Small sized GRU on this Hyperparameters
            embd_dim = 64
            hidden_dim = 128
            epochs = 35
            lr = 1e-3
            batch_size = 128
            block_size = 12
            dropout = 0.0
            optimizer = Adam
        """
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=n_classes,
            embedding_dim=embd_dim
        )

        self.gru = GRULayer(
            embd_dim=embd_dim,
            hidden_dim=hidden_dim
        )

        self.linear = Linear(
            hidden_dim=hidden_dim,
            n_classes=n_classes
        )

        self.n_classes = n_classes
        self.embd_dim = embd_dim
        self.hidden_dim = hidden_dim

    def forward(self, x):
        """
        x: (batch, seq_len) → indices of characters
        """

        # 1. Embedding: (batch, seq_len, embd_dim)
        x = self.embedding(x)

        # 2. GRU: (batch, seq_len, hidden_dim)
        h = self.gru(x)

        # 3. Linear projection: output logits
        logits = self.linear(h)   # (batch, seq_len, n_classes)

        return logits

In [7]:
class GRU_MediumModel(nn.Module):
    def __init__(self, n_classes, embd_dim, hidden_dim, dropout = 0.1):
        """
        I'll Train this Medium sized GRU on this Hyperparameters
            embd_dim = 96
            hidden_dim = 256
            epochs = 40
            lr = 5e-4
            batch_size = 128
            block_size = 16
            dropout = 0.1
            optimizer = Adam
        """
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=n_classes,
            embedding_dim=embd_dim
        )

        self.gru_layer_1 = GRULayer(
            embd_dim=embd_dim,
            hidden_dim=hidden_dim,
            dropout = dropout
        )

        # The output of previous gru_layer_1 is now the input of next gru_layer_2; that's why 'embd_dim = hidden_dim' for gru_layer_2
        self.gru_layer_2 = GRULayer(
            embd_dim = hidden_dim,
            hidden_dim = hidden_dim,
            dropout = dropout
        )

        self.linear = Linear(
            hidden_dim=hidden_dim,
            n_classes=n_classes
        )

        self.n_classes = n_classes
        self.embd_dim = embd_dim
        self.hidden_dim = hidden_dim
        self.dropout = dropout

    def forward(self, x):
        """
        x: (batch, seq_len) → indices of characters
        """

        # 1. Embedding: (batch, seq_len, embd_dim)
        x = self.embedding(x)

        # 2. GRU: (batch, seq_len, hidden_dim)
        h1 = self.gru_layer_1(x)
        h2 = self.gru_layer_2(h1)

        # 3. Linear projection: output logits
        logits = self.linear(h2)   # (batch, seq_len, n_classes)
        return logits

In [ ]:
class GRU_LargeModel(nn.Module):
    def __init__(self, n_classes, embd_dim, hidden_dim, dropout = 0.2):
        """
        I'll Train this Large sized GRU on this Hyperparameters
            embd_dim = 128
            hidden_dim = 512
            epochs = 60
            lr = 3e-4
            batch_size = 256
            block_size = 20
            dropout = 0.2
            optimizer = AdamW
        """
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=n_classes,
            embedding_dim=embd_dim
        )

        self.gru_layer_1 = GRULayer(
            embd_dim=embd_dim,
            hidden_dim=hidden_dim,
            dropout = dropout
        )
        self.gru_layer_2 = GRULayer(
            embd_dim = hidden_dim,
            hidden_dim = hidden_dim,
            dropout = dropout
        )
        self.gru_layer_3 = GRULayer(
            embd_dim = hidden_dim,
            hidden_dim = hidden_dim,
            dropout = dropout
        )

        self.norm = nn.LayerNorm(hidden_dim)

        self.linear = Linear(
            hidden_dim=hidden_dim,
            n_classes=n_classes
        )

        self.n_classes = n_classes
        self.embd_dim = embd_dim
        self.hidden_dim = hidden_dim

    def forward(self, x):
        """
        x: (batch, seq_len) → indices of characters
        """

        # 1. Embedding: (batch, seq_len, embd_dim)
        x = self.embedding(x)

        # 2. GRU: (batch, seq_len, hidden_dim) 
        h1 = self.gru_layer_1(x)
        h2 = self.gru_layer_2(h1)
        h3 = self.gru_layer_3(h2)

        # 3. LayerNorm Before Final Logits
        h3_norm = self.norm(h3)

        # 4. Linear projection: output logits
        logits = self.linear(h3_norm)   # (batch, seq_len, n_classes)

        return logits

In [9]:
MODEL_REGISTRY = {
    "GRU_SmallModel": GRU_SmallModel,
    "GRU_MediumModel": GRU_MediumModel,
    "GRU_LargeModel": GRU_LargeModel,
}

In [10]:
def print_model_summary(model, model_name, epochs, lr, device):
    print("\n" + "="*100)
    print("MODEL TRAINING CONFIGURATION")
    print("="*100)
    print(f"Model Name      : {model_name}")
    print(f"Device          : {device}")
    print(f"Total Epochs    : {epochs}")
    print(f"Learning Rate   : {lr}")
    
    print("\nMODEL ARCHITECTURE")
    print("-"*100)
    for name, module in model.named_modules():
        if name == "":
            print(f"  └── {module.__class__.__name__}()")
        else:
            print(f"  └── {name}: {module.__class__.__name__}()")
    print("-"*100)

    # Count parameters
    n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\nTrainable Parameters: {n_params:,}")
    print("="*100 + "\n")

In [11]:
def evaluate_validation_loss(model, val_loader, loss_fn, device):
    model.eval()     # IMPORTANT: evaluation mode
    total_loss = 0.0
    batches = 0

    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)

            logits = model(xb)

            loss = loss_fn(
                logits.reshape(-1, logits.size(-1)),
                yb.reshape(-1)
            )

            total_loss += loss.item()
            batches += 1

    model.train()   # switch back to training mode
    return total_loss / batches

In [12]:
def train_model(model, train_loader, val_loader, epochs, lr, device,
                optimizer, loss_fn, model_name):

    print_model_summary(model, model_name, epochs, lr, device)

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0.0
        last_logits = None

        for i, (xb, yb) in enumerate(train_loader):
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            logits = model(xb)

            # reshape for CE loss
            # NOTE: CrossEntropyLoss expects:
                    # logits: (N, C),  N --> number of Training samples --> batch_size, C --> n_classes
                    # targets: (N)
            
            # Our logits = model.forward(input); Have shape = (batch_size, seq_length, n_classes)
            # Cross Entropy extects, (training_examples, n_classes) so we should resize them
            # logits.reshape(-1, vocab_size) = (batch_size * seq_length, n_classes) or calculate 'vocab_size' dynamically as, logits.size(-1)
            # So, logits.reshape(-1, logits.size(-1)) --> logits.reshape(batch_size * seq_length, n_classes)

            loss = loss_fn(
                logits.reshape(-1, logits.size(-1)),
                yb.reshape(-1)
            )
            if i == 0:
                last_logits = logits.detach()

            loss.backward()

            optimizer.step()
            total_loss += loss.item()
        
        train_loss = total_loss / len(train_loader)
        val_loss = evaluate_validation_loss(model, val_loader, loss_fn, device)

        if epoch % 10 == 0:
            print(
                f"[Epoch {epoch:04d}/{epochs}] "
                f"Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | "
                f"LR: {optimizer.param_groups[0]['lr']:.6f}"
            )

## Function for Sampling from the Model

In [29]:
def generate_names(model, num_names, start_token, max_len, device):
    model.eval()
    results = []

    for i in range(1, num_names + 1):
        context = torch.tensor([[start_token]], device=device)  # (1, 1)

        for _ in range(max_len):
            logits = model(context)              # (1, seq_len, vocab)
            last_logits = logits[:, -1, :]       # last timestep
            probs = torch.softmax(last_logits, dim=-1)

            next_id = torch.multinomial(probs, num_samples=1).item()
            context = torch.cat([
                context,
                torch.tensor([[next_id]], device=device)
            ], dim=1)

            if next_id == 1:   # end token
                break

        # Decode
        token_ids = context[0].tolist()[1:]      # remove start token
        name = ''.join(itos[t] for t in token_ids)

        results.append(name)

        # Print with numeric prefix
        print(f"{i}. {name.capitalize()}")

    return results


## Function for Saving the Model

In [14]:
import os
import json
import torch
from datetime import datetime

def save_model(model, base_name="GRU", path="./saved_models/"):
    os.makedirs(path, exist_ok=True)

    # versioning
    existing = [f for f in os.listdir(path) if f.startswith(base_name) and f.endswith(".pth")]
    versions = []
    for f in existing:
        parts = f.replace(".pth", "").split("_v")
        if len(parts) == 2 and parts[1].isdigit():
            versions.append(int(parts[1]))
    next_version = max(versions, default=0) + 1

    filename = f"{base_name}_v{next_version}.pth"
    save_path = os.path.join(path, filename)

    checkpoint = {
        "state_dict": model.state_dict(),
        "model_class": model.__class__.__name__,
        "n_classes": model.n_classes,
        "embd_dim": model.embd_dim,
        "hidden_dim": model.hidden_dim,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "version": next_version,
    }

    torch.save(checkpoint, save_path)
    torch.save(checkpoint, os.path.join(path, f"{base_name}_latest.pth"))

    print(f"\nModel saved at: {save_path}")
    print(f"Also updated: {base_name}_latest.pth\n")
    return save_path


## Function for Loading the Model

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def load_model(filepath, device):
    checkpoint = torch.load(filepath, map_location=device)

    class_name = checkpoint["model_class"]
    if class_name not in MODEL_REGISTRY:
        raise ValueError(f"Unknown model class '{class_name}' in checkpoint.")

    ModelClass = MODEL_REGISTRY[class_name]

    # instantiate dynamically
    model = ModelClass(
        n_classes=checkpoint["n_classes"],
        embd_dim=checkpoint["embd_dim"],
        hidden_dim=checkpoint["hidden_dim"]
    ).to(device)

    model.load_state_dict(checkpoint["state_dict"])

    # print loaded model metadata
    print("\n================ MODEL LOADED ================")
    print(f"File: {filepath}")
    print(f"Model Class: {class_name}")
    print(f"Version: v{checkpoint['version']}")
    print(f"Timestamp: {checkpoint['timestamp']}")
    print("----------------------------------------------")

    print("Model Architecture:")
    for name, module in model.named_modules():
        if name != "":
            print(f"  └── {name}: {module.__class__.__name__}")

    print("----------------------------------------------")
    print(f"Total Trainable Parameters: {count_parameters(model):,}")
    print(f"Loaded on: {device}")
    print("==============================================\n")

    return model

##

<table style="width:100%; border-collapse:collapse; font-size:15px;" border="1">
  <tr>
    <th>Model Name</th>
    <th>Total Params</th>
    <th>Layers</th>
    <th>Embedding Dim (d<sub>embd</sub>)</th>
    <th>Hidden Dim (d<sub>hidden</sub>)</th>
    <th>Batch Size</th>
    <th>Block Size</th>
    <th>Dropout</th>
    <th>Learning Rate</th>
  </tr>

  <tr>
    <td>GRU-α (Small)</td>
    <td><i>0.080M</i></td>
    <td>1</td>
    <td>64</td>
    <td>128</td>
    <td>128</td>
    <td>12</td>
    <td>0.0</td>
    <td>\$(1.0 \times 10^{-3}$\)</td>
  </tr>

  <tr>
    <td>GRU-β (Medium)</td>
    <td><i>0.68M</i></td>
    <td>2</td>
    <td>96</td>
    <td>256</td>
    <td>128</td>
    <td>16</td>
    <td>0.1</td>
    <td><i></i></td>
  </tr>

  <tr>
    <td>GRU-γ (Large)</td>
    <td><i>4.16M</i></td>
    <td>3</td>
    <td>128</td>
    <td>512</td>
    <td>256</td>
    <td>20</td>
    <td>0.2</td>
    <td><i>3.0e−4</i></td>
  </tr>
</table>


## Defining 3 Different Architecture: Alpha, Beta, Gamma Models

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ---------------------------------------------------- Small GRU Model -------------------------------------------------------- 
small_model = GRU_SmallModel(
    n_classes = 28,
    embd_dim = 64,
    hidden_dim = 128,
    dropout = 0.0
).to(device)
small_model_name = "GRU_alpha_model"
lr_small_model = 1e-3
epochs_small_model = 35
optimizer_small = torch.optim.Adam(small_model.parameters(), lr = lr_small_model)


# ---------------------------------------------------- Medium GRU Model -------------------------------------------------------- 
medium_model = GRU_MediumModel(
    n_classes=28,
    embd_dim = 96,
    hidden_dim = 256,
    dropout = 0.1
).to(device)
medium_model_name = "GRU_beta_model"
lr_medium_model = 5e-4
epochs_medium_model = 40
optimizer_medium = torch.optim.Adam(medium_model.parameters(), lr = lr_medium_model)


# ---------------------------------------------------- Large GRU Model -------------------------------------------------------- 
large_model = GRU_LargeModel(
    n_classes=28,
    embd_dim = 128,
    hidden_dim = 512,
    dropout = 0.2
).to(device)
large_model_name = "GRU_gamma_model"
lr_large_model = 3e-4
epochs_large_model = 60
optimizer_large = torch.optim.AdamW(large_model.parameters(), lr = lr_large_model)


# ---------------------------------------------------- Model Registry -------------------------------------------------------- 
MODEL_REGISTRY = {
    "GRU_SmallModel": GRU_SmallModel,
    "GRU_MediumModel": GRU_MediumModel,
    "GRU_LargeModel": GRU_LargeModel,
}


# ---------------------------------------------------- Train and Validation Data Loader -------------------------------------------------------- 
loss_fn = nn.CrossEntropyLoss()
train_loader = DataLoader(
    TensorDataset(x_train, y_train),
    batch_size=64,
    shuffle = True
)

val_loader = DataLoader(
    TensorDataset(x_val, y_val),
    batch_size = 64,
    shuffle = False
)

## Training GRU Small Model: GRU_alpha_model

In [17]:
train_model(
    model=small_model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=epochs_small_model,
    lr=lr_small_model,
    device=device,
    optimizer=optimizer_small,
    loss_fn=loss_fn,
    model_name=small_model_name,
)


MODEL TRAINING CONFIGURATION
Model Name      : GRU_alpha_model
Device          : cuda
Total Epochs    : 35
Learning Rate   : 0.001

MODEL ARCHITECTURE
----------------------------------------------------------------------------------------------------
  └── GRU_SmallModel()
  └── embedding: Embedding()
  └── gru: GRULayer()
  └── gru.grucell: GRUCell()
  └── gru.grucell.Wx: Linear()
  └── gru.grucell.Wh: Linear()
  └── gru.grucell.Wzx: Linear()
  └── gru.grucell.Wzh: Linear()
  └── gru.grucell.Wrx: Linear()
  └── gru.grucell.Wrh: Linear()
  └── gru.dropout: Dropout()
  └── linear: Linear()
  └── linear.linear_projection: Linear()
----------------------------------------------------------------------------------------------------

Trainable Parameters: 79,772

[Epoch 0010/35] Train Loss: 1.7499 | Val Loss: 1.8252 | LR: 0.001000
[Epoch 0020/35] Train Loss: 1.6492 | Val Loss: 1.8151 | LR: 0.001000
[Epoch 0030/35] Train Loss: 1.5909 | Val Loss: 1.8292 | LR: 0.001000


In [30]:
generated = generate_names(
    model=small_model,
    num_names=20,
    start_token=0,
    max_len=20,
    device=device
)


1. Urt.
2. Elyann.
3. Ynner.
4. Ika.
5. Hidaleigh.
6. Ynzel.
7. Ynley.
8. Ella.
9. Iyana.
10. Oil.
11. Aylynn.
12. Handrea.
13. Ynishka.
14. Ayanne.
15. Ynnsleigh.
16. Inthim.
17. Ynnan.
18. Endy.
19. Ysen.
20. Hloei.


In [19]:
save_model(small_model, base_name="GRU_alpha_model")


Model saved at: ./saved_models/GRU_alpha_model_v1.pth
Also updated: GRU_alpha_model_latest.pth



'./saved_models/GRU_alpha_model_v1.pth'

## Train Medium Model: GRU_beta_model

In [20]:
train_model(
    model=medium_model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=epochs_medium_model,
    lr=lr_medium_model,
    device=device,
    optimizer=optimizer_medium,
    loss_fn=loss_fn,
    model_name = medium_model_name,
)


MODEL TRAINING CONFIGURATION
Model Name      : GRU_beta_model
Device          : cuda
Total Epochs    : 40
Learning Rate   : 0.0005

MODEL ARCHITECTURE
----------------------------------------------------------------------------------------------------
  └── GRU_MediumModel()
  └── embedding: Embedding()
  └── gru_layer_1: GRULayer()
  └── gru_layer_1.grucell: GRUCell()
  └── gru_layer_1.grucell.Wx: Linear()
  └── gru_layer_1.grucell.Wh: Linear()
  └── gru_layer_1.grucell.Wzx: Linear()
  └── gru_layer_1.grucell.Wzh: Linear()
  └── gru_layer_1.grucell.Wrx: Linear()
  └── gru_layer_1.grucell.Wrh: Linear()
  └── gru_layer_1.dropout: Dropout()
  └── gru_layer_2: GRULayer()
  └── gru_layer_2.grucell: GRUCell()
  └── gru_layer_2.grucell.Wx: Linear()
  └── gru_layer_2.grucell.Wh: Linear()
  └── gru_layer_2.grucell.Wzx: Linear()
  └── gru_layer_2.grucell.Wzh: Linear()
  └── gru_layer_2.grucell.Wrx: Linear()
  └── gru_layer_2.grucell.Wrh: Linear()
  └── gru_layer_2.dropout: Dropout()
  └── line

In [31]:
generated = generate_names(
    model=medium_model,
    num_names=20,
    start_token=0,
    max_len=20,
    device=device
)


1. .
2. Quinlynn.
3. Xander.
4. Suriah.
5. .
6. Karie.
7. Arona.
8. Xara.
9. Elodi.
10. Mannabelle.
11. Somani.
12. Attie.
13. Fadir.
14. Runiel.
15. Xaro.
16. Quest.
17. Samiya.
18. Braelynn.
19. Dannah.
20. Maddelene.


In [22]:
save_model(medium_model, base_name="GRU_beta_model")


Model saved at: ./saved_models/GRU_beta_model_v1.pth
Also updated: GRU_beta_model_latest.pth



'./saved_models/GRU_beta_model_v1.pth'

## Train Large Model: GRU_gamma_model

In [23]:
train_model(
    model=large_model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=epochs_large_model,
    lr=lr_large_model,
    device=device,
    optimizer=optimizer_large,
    loss_fn=loss_fn,
    model_name=large_model_name,
)


MODEL TRAINING CONFIGURATION
Model Name      : GRU_gamma_model
Device          : cuda
Total Epochs    : 60
Learning Rate   : 0.0003

MODEL ARCHITECTURE
----------------------------------------------------------------------------------------------------
  └── GRU_LargeModel()
  └── embedding: Embedding()
  └── gru_layer_1: GRULayer()
  └── gru_layer_1.grucell: GRUCell()
  └── gru_layer_1.grucell.Wx: Linear()
  └── gru_layer_1.grucell.Wh: Linear()
  └── gru_layer_1.grucell.Wzx: Linear()
  └── gru_layer_1.grucell.Wzh: Linear()
  └── gru_layer_1.grucell.Wrx: Linear()
  └── gru_layer_1.grucell.Wrh: Linear()
  └── gru_layer_1.dropout: Dropout()
  └── gru_layer_2: GRULayer()
  └── gru_layer_2.grucell: GRUCell()
  └── gru_layer_2.grucell.Wx: Linear()
  └── gru_layer_2.grucell.Wh: Linear()
  └── gru_layer_2.grucell.Wzx: Linear()
  └── gru_layer_2.grucell.Wzh: Linear()
  └── gru_layer_2.grucell.Wrx: Linear()
  └── gru_layer_2.grucell.Wrh: Linear()
  └── gru_layer_2.dropout: Dropout()
  └── gru_

In [33]:
generated = generate_names(
    model=large_model,
    num_names=20,
    start_token=0,
    max_len=20,
    device=device
)


1. Intly.
2. Inley.
3. Aili.
4. Aivah.
5. Adell.
6. Arris.
7. Oakley.
8. Aelen.
9. Ooa.
10. Ayari.
11. Ayaan.
12. Inam.
13. Aisaiah.
14. Aerin.
15. Inda.
16. Ira.
17. Iria.
18. Aiya.
19. Ona.
20. Inna.


In [25]:
save_model(large_model, base_name="GRU_gamma_model")



Model saved at: ./saved_models/GRU_gamma_model_v1.pth
Also updated: GRU_gamma_model_latest.pth



'./saved_models/GRU_gamma_model_v1.pth'

## Load All the Model's

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_model("./saved_models/GRU_alpha_model_v1.pth", device)



================ MODEL LOADED ================
File: ./saved_models/GRU_alpha_model_v1.pth
Model Class: GRU_SmallModel
Version: v1
Timestamp: 2025-12-11 20:34:52
----------------------------------------------
Model Architecture:
  └── embedding: Embedding
  └── gru: GRULayer
  └── gru.grucell: GRUCell
  └── gru.grucell.Wx: Linear
  └── gru.grucell.Wh: Linear
  └── gru.grucell.Wzx: Linear
  └── gru.grucell.Wzh: Linear
  └── gru.grucell.Wrx: Linear
  └── gru.grucell.Wrh: Linear
  └── gru.dropout: Dropout
  └── linear: Linear
  └── linear.linear_projection: Linear
----------------------------------------------
Total Trainable Parameters: 79,772
Loaded on: cuda



C:\Users\Himanshu Singh\AppData\Local\Temp\ipykernel_24668\2349754167.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath, map_location=devi

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_model("./saved_models/GRU_beta_model_v1.pth", device)


================ MODEL LOADED ================
File: ./saved_models/GRU_beta_model_v1.pth
Model Class: GRU_MediumModel
Version: v1
Timestamp: 2025-12-11 21:02:05
----------------------------------------------
Model Architecture:
  └── embedding: Embedding
  └── gru_layer_1: GRULayer
  └── gru_layer_1.grucell: GRUCell
  └── gru_layer_1.grucell.Wx: Linear
  └── gru_layer_1.grucell.Wh: Linear
  └── gru_layer_1.grucell.Wzx: Linear
  └── gru_layer_1.grucell.Wzh: Linear
  └── gru_layer_1.grucell.Wrx: Linear
  └── gru_layer_1.grucell.Wrh: Linear
  └── gru_layer_1.dropout: Dropout
  └── gru_layer_2: GRULayer
  └── gru_layer_2.grucell: GRUCell
  └── gru_layer_2.grucell.Wx: Linear
  └── gru_layer_2.grucell.Wh: Linear
  └── gru_layer_2.grucell.Wzx: Linear
  └── gru_layer_2.grucell.Wzh: Linear
  └── gru_layer_2.grucell.Wrx: Linear
  └── gru_layer_2.grucell.Wrh: Linear
  └── gru_layer_2.dropout: Dropout
  └── linear: Linear
  └── linear.linear_projection: Linear
-----------------------------------

C:\Users\Himanshu Singh\AppData\Local\Temp\ipykernel_24668\2349754167.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath, map_location=devi

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_model("./saved_models/GRU_gamma_model_v1.pth", device)


C:\Users\Himanshu Singh\AppData\Local\Temp\ipykernel_24668\2349754167.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath, map_location=devi


================ MODEL LOADED ================
File: ./saved_models/GRU_gamma_model_v1.pth
Model Class: GRU_LargeModel
Version: v1
Timestamp: 2025-12-11 22:10:22
----------------------------------------------
Model Architecture:
  └── embedding: Embedding
  └── gru_layer_1: GRULayer
  └── gru_layer_1.grucell: GRUCell
  └── gru_layer_1.grucell.Wx: Linear
  └── gru_layer_1.grucell.Wh: Linear
  └── gru_layer_1.grucell.Wzx: Linear
  └── gru_layer_1.grucell.Wzh: Linear
  └── gru_layer_1.grucell.Wrx: Linear
  └── gru_layer_1.grucell.Wrh: Linear
  └── gru_layer_1.dropout: Dropout
  └── gru_layer_2: GRULayer
  └── gru_layer_2.grucell: GRUCell
  └── gru_layer_2.grucell.Wx: Linear
  └── gru_layer_2.grucell.Wh: Linear
  └── gru_layer_2.grucell.Wzx: Linear
  └── gru_layer_2.grucell.Wzh: Linear
  └── gru_layer_2.grucell.Wrx: Linear
  └── gru_layer_2.grucell.Wrh: Linear
  └── gru_layer_2.dropout: Dropout
  └── gru_layer_3: GRULayer
  └── gru_layer_3.grucell: GRUCell
  └── gru_layer_3.grucell.Wx: Li

# **GRU Model Family — Architecture Summary**

| **Model Name**     | **nₚₐᵣₐₘₛ** | **nₗₐᵧₑᵣₛ** | **dₑₘᵦd** | **dₕᵢddₑₙ** | **Batch Size** | **Block Size** | **Dropout** | **Optimizer** |    **Learning Rate** |
| ------------------ | ----------: | ----------: | --------: | ----------: | -------------: | -------------: | ----------: | ------------- | -------------------: |
| **GRU-α (Small)**  |   **~180K** |       1 GRU |        64 |         128 |            128 |             12 |         0.0 | Adam          | (1.0 \times 10^{-3}) |
| **GRU-β (Medium)** |   **~520K** |       2 GRU |        96 |         256 |            128 |             16 |         0.1 | Adam          | (5.0 \times 10^{-4}) |
| **GRU-γ (Large)**  |   **~2.1M** |       3 GRU |       128 |         512 |            256 |             20 |         0.2 | AdamW         | (3.0 \times 10^{-4}) |

---

# **Notes on Table Design**

This table matches GPT-3’s formatting style:

* **Scientific notation** for learning rate.
* **Compact architecture descriptions** (similar to `d_model`, `n_layers`, `n_heads`).
* **Consistent numerical alignment** for research readability.
* **Greek-letter model names** mimic GPT naming conventions while staying unique.

---

# **Parameter Count Estimates**

Here is how approximate counts were derived:

### **1. GRU-α (Small)**

* embd_dim = 64
* hidden_dim = 128
* 1 GRU layer + embedding + linear
* Parameters ≈ **0.080M**

### **2. GRU-β (Medium)**

* embd_dim = 96
* hidden_dim = 256
* 2 stacked GRUs + embedding + linear
* Parameters ≈ **0.68M**

### **3. GRU-γ (Large)**

* embd_dim = 128
* hidden_dim = 512
* 3 stacked GRUs + embedding + linear + layernorm
* Parameters ≈ **4.16M**
